# LOCF IMPUTATION

In [ ]:
# Imports 
import pandas as pd
import numpy as np
import itertools 
import matplotlib.pyplot as plt
import copy

# Auxiliar functions
from aux import *

In [ ]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = 2000

In [ ]:
# Read all the data after fix: num_installs, macro android version,has, what's new, clean dev address
all_apps2= pd.read_csv("PATH.csv", delimiter="\t", encoding='utf-8', engine= 'python',converters={'_id':str})

### Obtain basic infomation

In [ ]:
clean_category = all_apps2.clean_category.unique()
top = all_apps2.top.unique()
countries = all_apps2.country.unique()

# Retrieve different dates as str
retrieved_date_data = all_apps2.retrieved_date_end.unique()
retrieved_date_data.sort()

# List combination between tops and category
clean_cat_no_editorChoice = np.delete(clean_category, np.where(clean_category == "editorChoice"))

list_combinations = [top[0:2], clean_cat_no_editorChoice]
top_category_combination = [p for p in itertools.product(*list_combinations)]

top_category_combination.append(('editorChoice','')) # list with all the possible top and category combinations

### Adding some auxiliar columns

In [ ]:
# Column to differentiate  general category vs all
condition = [all_apps2['clean_category']=='general']
output = [1]
all_apps2['General'] = pd.Series(np.select(condition, output, 0))

# A falg just for aggregations
all_apps2['Flag'] = 1 # just a number to flag the pivot tables
all_apps2['Delete']= 0 # 0 means that we do NOT need to delete this data
all_apps2['imputed'] = 0 # 0 means that is original data

### Data from missing weeks per country-category

In [ ]:
missing_apps_cat_country = pd.DataFrame(index= top_category_combination, dtype= str, columns= countries)
missing_num_cat_country = pd.DataFrame(index= top_category_combination, columns= countries, data= np.zeros((len(top_category_combination), len(countries))))

### Dictionaries by country - top and category

In [ ]:
#create a data frame dictionary to store data frames by countries
df_countries = {elem : pd.DataFrame for elem in countries}
for country in df_countries.keys():
    df_countries[country] = all_apps2[:][all_apps2.country == country]

In [ ]:
# Empty dictionaries
dict_co_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_usa_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_br_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_de_cate = {elem : pd.DataFrame for elem in top_category_combination}

In [ ]:
create_dataFrames_by_category(df_countries['co'], dict_co_cate)
create_dataFrames_by_category(df_countries['us'], dict_usa_cate)
create_dataFrames_by_category(df_countries['br'], dict_br_cate)
create_dataFrames_by_category(df_countries['de'], dict_de_cate)

In [ ]:
#BR
print('BR')
dict_pivot_cat_br2, incomplete_br =  pivot_sum_top(dict_br_cate,retrieved_date_data,'br',missing_num_cat_country, missing_apps_cat_country)
#CO
print('CO')
dict_pivot_cat_co2, incomplete_co =  pivot_sum_top(dict_co_cate,retrieved_date_data,'co',missing_num_cat_country,missing_apps_cat_country)
#DE
print('DE')
dict_pivot_cat_de2, incomplete_de =  pivot_sum_top(dict_de_cate,retrieved_date_data,'de',missing_num_cat_country, missing_apps_cat_country)
#US
print('US')
dict_pivot_cat_usa2, incomplete_usa =  pivot_sum_top(dict_usa_cate,retrieved_date_data,'us',missing_num_cat_country, missing_apps_cat_country)

In [ ]:
join_sumary_data = missing_num_cat_country.join(missing_apps_cat_country, lsuffix='_sum', rsuffix='_list')
#join_sumary_data.head(3)

### Dates for imputation

In [ ]:
dates = ['2017-11-11', '2017-11-18', '2017-11-25', '2017-12-02','2017-12-09', '2017-12-16', '2017-12-23', '2017-12-30','2018-01-06',
         '2018-01-13', '2018-01-20', '2018-01-27','2018-02-03', '2018-02-10', '2018-02-17', '2018-02-24','2018-03-03', '2018-03-10', 
         '2018-03-17', '2018-03-24','2018-03-31', '2018-04-07', '2018-04-14', '2018-04-21','2018-04-28', '2018-05-05', '2018-05-12', 
         '2018-05-19', '2018-05-26', '2018-06-02']
retrieved_date= ['2017-11-05', '2017-11-12', '2017-11-19', '2017-11-26',
       '2017-12-03', '2017-12-10', '2017-12-17', '2017-12-24',
       '2017-12-31', '2018-01-07', '2018-01-14', '2018-01-21',
       '2018-01-28', '2018-02-04', '2018-02-11', '2018-02-18',
       '2018-02-25', '2018-03-04', '2018-03-11', '2018-03-18',
       '2018-03-25', '2018-04-01', '2018-04-08', '2018-04-15',
       '2018-04-22', '2018-04-29', '2018-05-06', '2018-05-13',
       '2018-05-20', '2018-05-27'] 


In [ ]:
original_all_apps = copy.deepcopy(all_apps2)

In [ ]:
original_all_apps[original_all_apps.top=='editorChoice'].head(2)

In [ ]:
general_keep =join_sumary_data[(join_sumary_data.br_sum <=3) & (join_sumary_data.co_sum <=3) & (join_sumary_data.de_sum <=3) & (join_sumary_data.us_sum <=3)]
general_delete =join_sumary_data[(join_sumary_data.br_sum >3) | (join_sumary_data.co_sum >3) | (join_sumary_data.de_sum >3) | (join_sumary_data.us_sum >3)]
g_cat_keep= list(general_keep.index)
g_cat_delete= list(general_delete.index)

# per country
g_cat_br_delete = list(join_sumary_data[ (join_sumary_data.br_sum >3)].index)
g_cat_co_delete = list(join_sumary_data[ (join_sumary_data.co_sum >3)].index)
g_cat_usa_delete = list(join_sumary_data[ (join_sumary_data.us_sum >3)].index)
g_cat_de_delete = list(join_sumary_data[ (join_sumary_data.de_sum >3)].index)

In [ ]:
# Categories that we want to keep for analysis
for i in range(0,31):
    categories =join_sumary_data[(join_sumary_data.br_sum >i)|(join_sumary_data.co_sum >i)|(join_sumary_data.de_sum >i)|(join_sumary_data.us_sum >i)]
    num= categories.shape[0]
    total= join_sumary_data.shape[0] 
    print(f'for {i} we deleted {num}, {round((num/total)*100,2)}')
    print(f'         kept {total - num}, {round(((total -num)/total)*100,2)}')   
    
    if i >=1:
        print(list(categories.index))

In [ ]:
def app_imputation_LOCF(row,country,id_app,category,dates,all_apps,ii,retrieved_date):
    """Imputes data  WITH LOCF
    """
    imputed = 0
    new_data = []
    all_apps = all_apps.loc[(all_apps.id == id_app)& (all_apps.country==country)&(all_apps.top==category[0])]
    for i in range(0,len(row)):
        if np.isnan(row[i]):
            if i == 0:
                # if first week is not available then is going to be 0
                print("primera fila")
                row[i] = 0
            elif i >0:
                row[i] = row[i-1]
                if row[i] == 1:
                    #print(ii,id_app,category,country, dates[i])
                    if category[0] == '':
                        app = all_apps.loc[(all_apps.retrieved_date_end==dates[i-1])].copy()
                    else:
                        app = all_apps.loc[(all_apps.retrieved_date_end==dates[i-1]) & (all_apps.clean_category==category[1])].copy()
                    app['retrieved_date_end'] = dates[i]
                    app['retrieved_date_start'] = retrieved_date[i]


                    app['imputed'] = 1
                    imputed +=1 

                    all_apps = pd.concat([all_apps, app], sort=False,ignore_index=True, copy=False)
                    new_data.append(app)


    return row , new_data,imputed

## Colombia

In [ ]:
g_cat_co_delete

In [ ]:
print("++")
temp_all_apps = []
imputados= 0
for key, data in dict_pivot_cat_co2.items():
    #columns = dict_pivot_cat_co2[key][].columns
    
    if  key not in g_cat_co_delete:
        index = dict_pivot_cat_co2[key].index
        new_data = []
        
        for ind, value in enumerate(data[dates].values):
            row, new_all_apps,total = app_imputation_LOCF(value,'co',index[ind],key,dates,original_all_apps,ind,retrieved_date)
            new_data.append(row)
            imputados+=total
            if len(new_all_apps)>0:
                temp_all_apps.extend(new_all_apps)
        dict_pivot_cat_co2[key] = pd.DataFrame(columns=dates,data=new_data, index=index)
    print(key)

print("++")

In [ ]:
new_apps_co = pd.concat(temp_all_apps, sort=False,ignore_index=True)

In [ ]:
print(len(temp_all_apps),imputados,new_apps_co.shape)

In [ ]:
new_apps_co.shape

## Brazil

In [ ]:
g_cat_br_delete

In [ ]:
print(original_all_apps.shape)
print("++")
temp_all_apps = []
imputados= 0
for key, data in dict_pivot_cat_br2.items():
    
    if  key not in g_cat_br_delete:
        index = dict_pivot_cat_br2[key].index
        new_data = []
        
        for ind, value in enumerate(data[dates].values):
            row, new_all_apps,total = app_imputation_LOCF(value,'br',index[ind],key,dates,original_all_apps,ind,retrieved_date)
            new_data.append(row)
            imputados+=total
            if len(new_all_apps)>0:
                temp_all_apps.extend(new_all_apps)
        dict_pivot_cat_br2[key] = pd.DataFrame(columns=dates,data=new_data, index=index)
    print(key)

print("++")
print(original_all_apps.shape)

In [ ]:
print(len(temp_all_apps),imputados)

In [ ]:
if len(temp_all_apps)>0:
    new_apps_br = pd.concat(temp_all_apps, sort=False,ignore_index=True)
    print(len(temp_all_apps),imputados,new_apps_usa.shape)

## USA

In [ ]:
g_cat_usa_delete

In [ ]:
print("++")
temp_all_apps = []
imputados= 0
for key, data in dict_pivot_cat_usa2.items():
    
    if  key not in g_cat_usa_delete:
        index = dict_pivot_cat_usa2[key].index
        new_data = []
        
        for ind, value in enumerate(data[dates].values):
            row, new_all_apps,total = app_imputation_LOCF(value,'us',index[ind],key,dates,original_all_apps,ind,retrieved_date)
            new_data.append(row)
            imputados+=total
            if len(new_all_apps)>0:
                temp_all_apps.extend(new_all_apps)
        dict_pivot_cat_usa2[key] = pd.DataFrame(columns=dates,data=new_data, index=index)
    print(key)

print("++")

In [ ]:
print(len(temp_all_apps),imputados)

In [ ]:
if len(temp_all_apps)>0:
    new_apps_usa = pd.concat(temp_all_apps, sort=False,ignore_index=True)
    print(len(temp_all_apps),imputados,new_apps_usa.shape)

## Germany

In [ ]:
g_cat_de_delete

In [ ]:
print("++")
temp_all_apps = []
imputados= 0
for key, data in dict_pivot_cat_de2.items():
    
    if  key not in g_cat_de_delete:
        index = dict_pivot_cat_de2[key].index
        new_data = []
        
        for ind, value in enumerate(data[dates].values):
            row, new_all_apps,total = app_imputation_LOCF(value,'de',index[ind],key,dates,original_all_apps,ind,retrieved_date)
            new_data.append(row)
            imputados+=total
            if len(new_all_apps)>0:
                temp_all_apps.extend(new_all_apps)
        dict_pivot_cat_de2[key] = pd.DataFrame(columns=dates,data=new_data, index=index)
    print(key)

print("++")

In [ ]:
print(len(temp_all_apps),imputados)

In [ ]:
if len(temp_all_apps)>0:
    new_apps_de = pd.concat(temp_all_apps, sort=False,ignore_index=True)
    print(len(temp_all_apps),imputados,new_apps_de.shape)

In [ ]:
all_apps3 = pd.concat([new_apps_de,new_apps_co,original_all_apps], sort=False,ignore_index=True)
all_apps3.shape, original_all_apps.shape

In [ ]:
g_cat_delete

In [ ]:
g_cat_co_delete

In [ ]:
g_cat_br_delete

In [ ]:
g_cat_usa_delete

In [ ]:
g_cat_de_delete

In [ ]:
all_apps3['Delete'] = all_apps3[['clean_category','top']].apply (lambda row: delete_flag(row, g_cat_delete), axis=1)
all_apps3['Delete_bo'] = all_apps3[['clean_category','top']].apply (lambda row: delete_flag(row, g_cat_co_delete), axis=1)
all_apps3['Delete_br'] = all_apps3[['clean_category','top']].apply (lambda row: delete_flag(row, g_cat_br_delete), axis=1)
all_apps3['Delete_de'] = all_apps3[['clean_category','top']].apply (lambda row: delete_flag(row, g_cat_de_delete), axis=1)
all_apps3['Delete_usa'] = all_apps3[['clean_category','top']].apply (lambda row: delete_flag(row, g_cat_usa_delete), axis=1)

In [ ]:
all_apps3.to_csv('data_LOCF.csv', sep='\t', index=False)